In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from scipy import stats
from scipy.stats import boxcox
import ipywidgets as widgets
import statsmodels.api as sm
from statsmodels.tsa.seasonal import STL
import pip
import itertools 
import warnings
from statsmodels.tsa.exponential_smoothing.ets import ETSModel
from statsmodels.tsa.api import ExponentialSmoothing
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
import matplotlib.ticker as ticker

In [2]:
#Getting consumer price data from January 2015 to April 2022

df = pd.read_excel(r'C:\Users\Alina\Desktop\Cases\Prices.xlsx', engine = 'openpyxl', index_col = 'Data')
df = df.transpose()

In [3]:
df.head(2)

Data,Margarine,Sour cream,Beef (except boneless meat),"Whole pasteurized drinking milk 2,5-3,2% fat content","Sterilized whole drinking milk 2,5-3,2% fat content",Pork (except boneless meat),Chickens,Lamb (except boneless meat),Chesses hard and soft,Eggs,...,Potatoes,Fresh cabbage,Onions,Carrots,Cucumbers,Tomatoes,Apples,Vodka,Fish,Butter
2015-01-04 00:00:00,89.18,156.790,277.430,43.950,59.85,274.93,137.390,309.250,395.540,60.79,...,28.01,28.72,27.77,32.710,137.920,138.29,80.79,548.160,113.280,360.430
2015-01-11 00:00:00,89.88,157.755,280.045,44.085,60.05,275.88,137.995,310.345,400.355,61.53,...,28.87,30.71,29.07,33.965,150.605,144.19,82.44,548.905,114.295,362.825


# Comparison of models

In [ ]:
#Comparison of models to identify seasonality in prices

In [4]:
all_df1 = pd.DataFrame()
all_season1 = pd.DataFrame()
all_trend1 = pd.DataFrame()
all_err1 = pd.DataFrame()
all_dol_season1 = pd.DataFrame()
all_df2 = pd.DataFrame()
all_season2 = pd.DataFrame()
all_trend2 = pd.DataFrame()
all_err2 = pd.DataFrame()
all_dol_season2 = pd.DataFrame()
all_df3 = pd.DataFrame()
all_season3 = pd.DataFrame()
all_trend3 = pd.DataFrame()
all_err3 = pd.DataFrame()
all_dol_season3 = pd.DataFrame()
spisok_col = sorted(df.columns.astype(str))
for a in spisok_col:
    result1 = sm.tsa.seasonal_decompose(df[a], model='additive', period=52, extrapolate_trend=1) #Model 1. Additive seasonal 
    new_err1 = np.mean(np.abs(df[a] - result1.seasonal - result1.trend)) #Calculation of MAE
    new_dol_season1 = result1.seasonal/(df[a]-result1.seasonal) #Calculation of the contribution of seasonality to the prices
    new_df1 = pd.DataFrame({a: df[a], a + ' Trend': result1.trend, 
                            a + ' Seasonality': result1.seasonal, a + ' MAE': new_err1, a + ' Seasonality share': new_dol_season1}) # отображение результатов
    season1 = pd.DataFrame({a + ' Seasonality': result1.seasonal})
    trend1 = pd.DataFrame({a + ' Trend': result1.trend})
    err1 = pd.DataFrame({a + ' MAE': [new_err1]})
    dol_season1 = pd.DataFrame({a + ' Seasonality share': [new_dol_season1]})
    
    result2 = sm.tsa.seasonal_decompose(df[a], model='multiplicative', period=52, extrapolate_trend=1) #Model 2. Multiplicative seasonal
    new_err2 = np.mean(np.abs(df[a] / (result2.seasonal*result2.trend)))
    new_dol_season2 = result2.seasonal/(df[a]-result2.seasonal) 
    new_df2 = pd.DataFrame({a: df[a], a + ' Tend': result2.trend, 
                            a + ' Seasonality': result2.seasonal, a + ' MAE': new_err2, a + ' Seasonality share': new_dol_season2}) # отображение результатов
    season2 = pd.DataFrame({a + ' Seasonality': result2.seasonal})
    trend2 = pd.DataFrame({a + ' Trend': result2.trend})
    err2 = pd.DataFrame({a + ' MAE': [new_err2]})
    dol_season2 = pd.DataFrame({a + ' Seasonality share': [new_dol_season2]})
    
    stl = STL(df[a], period=52, robust=True) #Model 3. STL
    result3 = stl.fit()
    new_err3 = np.mean(np.abs(df[a] - result3.seasonal - result3.trend)) 
    new_dol_season3 = result3.seasonal/(df[a]-result3.seasonal)
    new_df3 = pd.DataFrame({a: df[a], a + ' Trend': result3.trend, 
                            a + ' Seasonality': result3.seasonal, a + ' MAE': new_err3, a + ' Seasonality share': new_dol_season2}) # отображение результатов
    season3 = pd.DataFrame({a + ' Seasonality': result3.seasonal})
    trend3 = pd.DataFrame({a + ' Trend': result3.trend})
    err3 = pd.DataFrame({a + ' MAE': [new_err3]})
    dol_season3 = pd.DataFrame({a + ' Seasonality share': [new_dol_season3]})
    
    minimum = np.minimum(new_err1, np.minimum(new_err2, new_err3)) #Calculation of minimum MAE in the results of all models
    m = pd.DataFrame({a + ' Min error': [minimum]})
    #Сравнение моделей
    if np.equal([m], [err1]):
        all_df1 = pd.concat([all_df1, new_df1], axis=1)
        all_season1 = pd.concat([all_season1, season1], axis=1)
        all_trend1 = pd.concat([all_trend1,  trend1], axis=1)
        all_err1 = pd.concat([all_err1,  err1], axis=1)
        all_dol_season1 = pd.concat([all_dol_season1, dol_season1], axis=1)
    elif np.equal([m], [err2]):
        all_df2 = pd.concat([all_df2, new_df2], axis=1)
        all_season2 = pd.concat([all_season2, season2], axis=1)
        all_trend2 = pd.concat([all_trend2, trend2], axis=1)
        all_err2 = pd.concat([all_err2,  err2], axis=1)
        all_dol_season2 = pd.concat([all_dol_season2, dol_season2], axis=1)
    else:
        all_df3 = pd.concat([all_df3, new_df3], axis=1)
        all_season3 = pd.concat([all_season3, season3], axis=1)
        all_trend3 = pd.concat([all_trend3,  trend3], axis=1)
        all_err3 = pd.concat([all_err3,  err3], axis=1)
        all_dol_season3 = pd.concat([all_dol_season3, dol_season3], axis=1)

#Saving results
output = pd.ExcelWriter('Results.xlsx')
all_df1.to_excel(output, sheet_name='Model 1')
all_df2.to_excel(output, sheet_name='Model 2')
all_df3.to_excel(output, sheet_name='Model 3')
output.save()

# Forecast

In [ ]:
#forecast on the base of seasonality contribution and stationarity tests using ETS model

In [5]:
all_df4 = pd.DataFrame()
all_df4 = pd.concat([all_df1, all_df2, all_df3, all_df4], axis=1)

In [6]:
warnings.filterwarnings("ignore")

y1 = pd.DataFrame()
y2 = pd.DataFrame()
y3 = pd.DataFrame()
y4 = pd.DataFrame()
y5 = pd.DataFrame()
y6 = pd.DataFrame()
for a in spisok_col:
    test1 = sm.tsa.adfuller(df[a], regression='c') #Dickey-Fuller stationarity test
    s_mean = np.mean(abs(all_df4[a+' Seasonality share'])) #Calculation of average seasonality contribution
    if test1[0] < test1[4]['5%'] and s_mean < 0.1: 
        mod1 = ExponentialSmoothing(df[a], use_boxcox=True, initialization_method="estimated").fit() #If time series are stationary and seasonality contribution is small, model is used without additional parameters
        pred1 = mod1.predict(start=1, end=378) #forecast of prices
        new_y1 = pd.DataFrame({a: pred1})
        y1 = pd.concat([y1, new_y1], axis=1)
        print(a)
        print(pred1)
    elif test1[0] < test1[4]['5%'] and s_mean >= 0.1:
        mod2 = ExponentialSmoothing(df[a], seasonal_periods=52,  seasonal="add", use_boxcox=True, initialization_method="estimated").fit() #If time series are stationary and seasonality contribution is large, model is used with seasonal parameter
        pred2 = mod2.predict(start=1, end=378) #forecast of prices
        new_y2 = pd.DataFrame({a: pred2})
        y2 = pd.concat([y2, new_y2], axis=1)
        print(a)
        print(pred2)
    else:
        test2 = sm.tsa.adfuller(df[a], regression='ctt') #Dickey-Fuller test to check of trend stationarity
        if test2[0] < test2[4]['5%'] and s_mean < 0.1:
            mod3 = ExponentialSmoothing(df[a], trend="add", use_boxcox=True, initialization_method="estimated").fit() #If time series have trend stationarity and seasonality contribution is small, model is used with trend component
            pred3 = mod3.predict(start=1, end=378) #forecast of prices
            new_y3 = pd.DataFrame({a: pred3})
            y3 = pd.concat([y3, new_y3], axis=1)
            print(a)
            print(pred3)
        elif test2[0] < test2[4]['5%'] and s.mean >= 0.1:
            mod4 = ExponentialSmoothing(df[a], seasonal_periods=52, trend="add", seasonal="add", use_boxcox=True, initialization_method="estimated").fit() #If time series have trend stationarity and seasonality contribution is large, model is used with both trend and seasonal components
            pred4 = mod4.predict(start=1, end=378) #forecast of prices
            new_y4 = pd.DataFrame({a: pred4})
            y4 = pd.concat([y4, new_y4], axis=1)
            print(a)
            print(pred4)
        elif test2[0] > test2[4]['5%'] and s_mean < 0.1:
            Dif = df[a] - df[a].shift(1) #If time series are fully stationary, differences are taken
            D = Dif.fillna(0)
            mod5 = ExponentialSmoothing(D, trend="add", initialization_method="estimated").fit() #model is used with trend component, but without seasonal component due to small seasonality contribution
            pred5 = mod5.predict(start=1, end=378) #forecast of prices differences
            pred5_1 = pd.Series(pred5, index=df.index)
            new_y5 = pd.DataFrame({a + ' Differences forecast': pred5_1})
            y5 = pd.concat([y5,  new_y5], axis=1)
            print(a)
            print(pred5_1)
        else:
            mod6 = ExponentialSmoothing(D, seasonal_periods=52, trend="add", seasonal="add", initialization_method="estimated").fit() #model is used with both seasonal and trend components
            pred6 = mod6.predict(start=1, end=378) #forecast of differences
            pred6_1 = pd.Series(pred6, index=df.index) 
            new_y6 = pd.DataFrame({a + ' Differences forecast': pred6_1})
            y6 = pd.concat([y6,  new_y6], axis=1)
            print(a)
            print(pred6_1)
            
output = pd.ExcelWriter('Differences ETS.xlsx')
y5.to_excel(output, sheet_name='Differences 1')
y6.to_excel(output, sheet_name='Differences 2')
output.save()

Apples
2015-01-11 00:00:00     80.790863
2015-01-18 00:00:00     82.431479
2015-01-25 00:00:00     84.081434
2015-02-01 00:00:00     86.387882
2015-02-08 00:00:00     88.906730
                          ...    
11.03.2022             105.921688
18.03.2022             108.675565
25.03.2022             112.857637
01.04.2022             117.047643
08.04.2022             121.158137
Length: 378, dtype: float64
Beef (except boneless meat)
2015-01-04 00:00:00         NaN
2015-01-11 00:00:00    1.052330
2015-01-18 00:00:00    1.673462
2015-01-25 00:00:00    2.050896
2015-02-01 00:00:00    2.592160
                         ...   
11.03.2022             2.664498
18.03.2022             3.129803
25.03.2022             5.574374
01.04.2022             7.283465
08.04.2022             6.336729
Length: 379, dtype: float64
Biscuits
2015-01-04 00:00:00         NaN
2015-01-11 00:00:00    0.095970
2015-01-18 00:00:00    0.507053
2015-01-25 00:00:00    0.646072
2015-02-01 00:00:00    1.043851
              

Potatoes
2015-01-11 00:00:00    28.475810
2015-01-18 00:00:00    29.780456
2015-01-25 00:00:00    30.646703
2015-02-01 00:00:00    32.444172
2015-02-08 00:00:00    33.063983
                         ...    
11.03.2022             48.851684
18.03.2022             50.435775
25.03.2022             52.610996
01.04.2022             55.366869
08.04.2022             58.240857
Length: 378, dtype: float64
Rice
2015-01-04 00:00:00         NaN
2015-01-11 00:00:00    0.072435
2015-01-18 00:00:00    0.879350
2015-01-25 00:00:00    1.072904
2015-02-01 00:00:00    1.465191
                         ...   
11.03.2022             0.760323
18.03.2022             2.051526
25.03.2022             3.227795
01.04.2022             3.297109
08.04.2022             3.184513
Length: 379, dtype: float64
Salt
2015-01-04 00:00:00         NaN
2015-01-11 00:00:00    0.010416
2015-01-18 00:00:00    0.017980
2015-01-25 00:00:00    0.022173
2015-02-01 00:00:00    0.062382
                         ...   
11.03.2022        

In [7]:
df_dif = pd.read_excel(r'C:\Users\Alina\Desktop\Cases\Forecast prices by differences.xlsx', engine = 'openpyxl', index_col = 'Data')
df_dif.head(2)

,Beef (except boneless meat),Biscuits,Black tea,Boiled sausage,Bread,Buckwheat-kernel groats,Butter,Chesses hard and soft,Chickens,Fresh cabbage,...,Rice,Salt,Sausages,Semi-smoked and boiled-smoked sausage,Sour cream,"Sterilized whole drinking milk 2,5-3,2% fat content",Sweets,Vodka,Wheat flour,"Whole pasteurized drinking milk 2,5-3,2% fat content"
Data,,,,,,,,,,,,,,,,,,,,,
2015-01-04 00:00:00,309.250000,56.980000,548.160000,277.43000,28.720000,311.520000,373.830000,216.660000,68.130000,137.39000,...,33.010000,55.090000,48.570000,274.930000,156.790000,11.620000,295.740000,395.540000,37.270000,506.800000
2015-01-11 00:00:00,310.665925,57.195125,548.111104,278.48233,28.725397,312.380189,375.135426,217.527624,68.208243,137.39778,...,33.146523,55.162435,48.571931,274.940744,157.174218,11.630416,296.449053,397.288382,37.381024,508.618858


In [8]:
#Combining all forecasts
ETS = pd.concat([y1,y2,y3,y4,df_dif], axis=1)

In [9]:
#calculation of MAE
df_true = df.drop(pd.to_datetime('2015-01-04 00:00:00'))
df_fs = ETS.drop(pd.to_datetime('2015-01-04 00:00:00'))
spisok_col = sorted(df_fs.columns.astype(str))
for a in spisok_col:
    err_ETS = mean_absolute_error(df_true[a], df_fs[a])
    print(a)
    print(err_ETS)

Apples
1.0529782162191468
Beef (except boneless meat)
57.36536116675475
Biscuits
84.65419532325048
Black tea
190.82752817952726
Boiled sausage
33.6039749214172
Bread
19.72832247896435
Buckwheat-kernel groats
309.2180428514545
Butter
79.27099399021516
Carrots
1.056759695056528
Chesses hard and soft
204.1079498980967
Chickens
66.71689119952532
Cucumbers
5.305396897517673
Eggs
0.27895389576230023
Fish
0.27151085663330154
Fresh cabbage
114.39553550531332
Granulated sugar
27.989322276876543
Lamb (except boneless meat)
253.63040046621944
Margarine
405.42459449089984
Millet
11.922597335025419
Onions
0.6678493028338294
Pasta
4.828683354377877
Pasta from wheat flour of the highest grade
36.885102886533524
Pork (except boneless meat)
113.01485014380002
Potatoes
0.24282464557378358
Rice
21.737602170047634
Salt
58.78967810966095
Sausages
309.9180819031961
Semi-smoked and boiled-smoked sausage
190.0273479869367
Sour cream
1.4487660750103266
Sterilized whole drinking milk 2,5-3,2% fat content
60.308

In [12]:
for a in spisok_col:
    MAPE = mean_absolute_percentage_error(df_true[a], df_fs[a])
    print(a)
    print(MAPE)

Apples
0.010268025067093815
Beef (except boneless meat)
0.16623278409970446
Biscuits
0.5472722203064339
Black tea
0.23086573771803395
Boiled sausage
0.08945558532781481
Bread
0.40323443727693253
Buckwheat-kernel groats
4.489371153274965
Butter
0.13563680469648873
Carrots
0.024596447191539875
Chesses hard and soft
0.3949930216010217
Chickens
0.47055846527194267
Cucumbers
0.04687605442684642
Eggs
0.004418334835373007
Fish
0.001712691337859211
Fresh cabbage
4.4458798668685215
Granulated sugar
0.6459173671728804
Lamb (except boneless meat)
0.6517696370668569
Margarine
3.056350000055991
Millet
0.2911667818762409
Onions
0.020825826733868586
Pasta
0.06917816742323052
Pasta from wheat flour of the highest grade
0.502898862928049
Pork (except boneless meat)
0.41794514447176007
Potatoes
0.00757620943916268
Rice
0.3168615143541977
Salt
4.439480626688585
Sausages
0.8672210212404007
Semi-smoked and boiled-smoked sausage
0.4083013582976451
Sour cream
0.006950888857082543
Sterilized whole drinking mi

In [ ]:
#The results show a huge variation in errors. If for some products (apples or fish) the errors are 1%, then for others the difference can be two times, which is a poor indicator for the model. One could analyze the reasons why the model works for some prices and not for others, but we have model that can predict better for all goods thanks to more fine-tuning of parameters for each product.